# 📝 Word-Level vs Sentence-Level Accent Detection

## Objective
Compare accent classification performance using word-level vs sentence-level speech segments.

## Research Question
Which speech unit provides stronger and more stable accent cues?

## Methodology
1. Segment audio into word-level and sentence-level chunks
2. Extract features from both
3. Train separate models on each
4. Compare accuracy, robustness, and interpretability

## Expected Outcome
- Sentence-level expected to have higher accuracy (more context)
- Word-level may be more robust to noise
- Analysis of minimum speech duration needed for reliable detection

## Estimated Runtime: 4-5 hours

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_DIR = '/content/drive/MyDrive/IndicAccent_Project'
os.chdir(PROJECT_DIR)
print(f'✅ Working directory: {os.getcwd()}')

In [ ]:
!pip install -q datasets transformers torch torchaudio librosa soundfile scikit-learn matplotlib tqdm pydub
!pip install -q torchcodec
print('✅ Dependencies installed!')

## Step 1: Load Dataset

In [ ]:
from datasets import load_dataset, Audio
import numpy as np
import librosa

dataset = load_dataset("DarshanaS/IndicAccentDb")
dataset = dataset.cast_column("audio", Audio(decode=True))
dataset = dataset.shuffle(seed=42)

print(f'Total samples: {len(dataset["train"])}')

## Step 2: Segment Audio

### Word-Level Segmentation
- Use Voice Activity Detection (VAD) to find speech segments
- Split into ~1-2 second chunks (typical word duration)

### Sentence-Level Segmentation
- Use longer segments (~5-10 seconds)
- Or use full utterances if available

In [ ]:
def segment_audio_by_duration(audio_array, sr, segment_duration):
    """
    Segment audio into fixed-duration chunks
    
    Args:
        audio_array: Audio signal
        sr: Sample rate
        segment_duration: Duration in seconds
    
    Returns:
        List of audio segments
    """
    segment_samples = int(segment_duration * sr)
    segments = []
    
    for start in range(0, len(audio_array), segment_samples):
        end = min(start + segment_samples, len(audio_array))
        segment = audio_array[start:end]
        
        # Only keep segments that are at least 50% of target duration
        if len(segment) >= segment_samples * 0.5:
            # Pad if necessary
            if len(segment) < segment_samples:
                segment = np.pad(segment, (0, segment_samples - len(segment)))
            segments.append(segment)
    
    return segments

def create_word_level_dataset(dataset, word_duration=1.5):
    """
    Create word-level dataset by segmenting audio
    """
    word_segments = []
    word_labels = []
    
    for item in dataset:
        audio = item['audio']
        arr = audio['array'].astype(float)
        sr = audio['sampling_rate']
        label = item['label']
        
        # Resample to 16kHz
        if sr != 16000:
            arr = librosa.resample(arr, orig_sr=sr, target_sr=16000)
            sr = 16000
        
        # Segment
        segments = segment_audio_by_duration(arr, sr, word_duration)
        
        # Add all segments with same label
        for seg in segments:
            word_segments.append(seg)
            word_labels.append(label)
    
    return word_segments, word_labels

def create_sentence_level_dataset(dataset, sentence_duration=5.0):
    """
    Create sentence-level dataset
    """
    sentence_segments = []
    sentence_labels = []
    
    for item in dataset:
        audio = item['audio']
        arr = audio['array'].astype(float)
        sr = audio['sampling_rate']
        label = item['label']
        
        # Resample to 16kHz
        if sr != 16000:
            arr = librosa.resample(arr, orig_sr=sr, target_sr=16000)
            sr = 16000
        
        # If audio is shorter than sentence duration, pad it
        target_samples = int(sentence_duration * sr)
        if len(arr) < target_samples:
            arr = np.pad(arr, (0, target_samples - len(arr)))
        else:
            # Take first sentence_duration seconds
            arr = arr[:target_samples]
        
        sentence_segments.append(arr)
        sentence_labels.append(label)
    
    return sentence_segments, sentence_labels

print('✅ Segmentation functions defined')

In [ ]:
from tqdm import tqdm

# Use subset for demonstration (remove limit for full analysis)
subset = dataset['train'].select(range(min(1000, len(dataset['train']))))

print("Creating word-level dataset (1.5s segments)...")
word_segments, word_labels = create_word_level_dataset(subset, word_duration=1.5)
print(f"✅ Word-level: {len(word_segments)} segments")

print("\nCreating sentence-level dataset (5.0s segments)...")
sentence_segments, sentence_labels = create_sentence_level_dataset(subset, sentence_duration=5.0)
print(f"✅ Sentence-level: {len(sentence_segments)} segments")

print(f"\n📊 Comparison:")
print(f"   Word-level: {len(word_segments)} samples (avg {len(word_segments)/len(subset):.1f} per utterance)")
print(f"   Sentence-level: {len(sentence_segments)} samples (1 per utterance)")

## Step 3: Extract Features

In [ ]:
import torch
from transformers import HubertModel, Wav2Vec2FeatureExtractor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load HuBERT
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
hubert = HubertModel.from_pretrained("facebook/hubert-base-ls960").to(device)
hubert.eval()

def extract_mfcc_from_array(arr, sr=16000):
    arr = arr / (np.max(np.abs(arr)) + 1e-9)
    mfcc = librosa.feature.mfcc(y=arr, sr=sr, n_mfcc=40)
    return np.concatenate([mfcc.mean(axis=1), mfcc.std(axis=1)])

def extract_hubert_from_array(arr, sr=16000):
    arr = arr / (np.max(np.abs(arr)) + 1e-9)
    inputs = feature_extractor(arr, sampling_rate=sr, return_tensors="pt", padding=True)
    
    with torch.no_grad():
        outputs = hubert(inputs.input_values.to(device))
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy()[0]
    
    return embedding

print('✅ Feature extraction ready')

In [ ]:
# Extract features for word-level
print("Extracting WORD-LEVEL features...")

X_word_mfcc = []
X_word_hubert = []

for seg in tqdm(word_segments):
    try:
        X_word_mfcc.append(extract_mfcc_from_array(seg))
        X_word_hubert.append(extract_hubert_from_array(seg))
    except:
        continue

X_word_mfcc = np.array(X_word_mfcc)
X_word_hubert = np.array(X_word_hubert)
y_word = np.array(word_labels[:len(X_word_mfcc)])

print(f"✅ Word features: MFCC {X_word_mfcc.shape}, HuBERT {X_word_hubert.shape}")

In [ ]:
# Extract features for sentence-level
print("Extracting SENTENCE-LEVEL features...")

X_sent_mfcc = []
X_sent_hubert = []

for seg in tqdm(sentence_segments):
    try:
        X_sent_mfcc.append(extract_mfcc_from_array(seg))
        X_sent_hubert.append(extract_hubert_from_array(seg))
    except:
        continue

X_sent_mfcc = np.array(X_sent_mfcc)
X_sent_hubert = np.array(X_sent_hubert)
y_sent = np.array(sentence_labels[:len(X_sent_mfcc)])

print(f"✅ Sentence features: MFCC {X_sent_mfcc.shape}, HuBERT {X_sent_hubert.shape}")

## Step 4: Train and Compare

In [ ]:
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class Classifier(nn.Module):
    def __init__(self, input_dim, num_classes=6):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.net(x)

class FeatureDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, i):
        return self.X[i], self.y[i]

def train_model(X, y, input_dim, model_name):
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    
    # Split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # Dataloaders
    train_loader = DataLoader(FeatureDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(FeatureDataset(X_val, y_val), batch_size=64)
    
    # Model
    model = Classifier(input_dim).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    # Train
    best_acc = 0
    for epoch in range(15):
        model.train()
        for feats, labels in train_loader:
            feats, labels = feats.to(device), labels.to(device)
            optimizer.zero_grad()
            preds = model(feats)
            loss = loss_fn(preds, labels)
            loss.backward()
            optimizer.step()
        
        # Validate
        model.eval()
        correct = total = 0
        with torch.no_grad():
            for feats, labels in val_loader:
                feats, labels = feats.to(device), labels.to(device)
                preds = model(feats).argmax(dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        acc = correct / total
        if acc > best_acc:
            best_acc = acc
        print(f"Epoch {epoch+1}/15 | Val Acc: {acc:.4f}")
    
    print(f"✅ Best Accuracy: {best_acc:.4f}")
    return best_acc

print('✅ Training function ready')

In [ ]:
# Train all combinations
results = {}

results['Word-MFCC'] = train_model(X_word_mfcc, y_word, 80, "Word-Level MFCC")
results['Word-HuBERT'] = train_model(X_word_hubert, y_word, 768, "Word-Level HuBERT")
results['Sentence-MFCC'] = train_model(X_sent_mfcc, y_sent, 80, "Sentence-Level MFCC")
results['Sentence-HuBERT'] = train_model(X_sent_hubert, y_sent, 768, "Sentence-Level HuBERT")

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
for name, acc in results.items():
    print(f"{name:20s}: {acc:.4f}")

## Step 5: Visualize Comparison

In [ ]:
import matplotlib.pyplot as plt

# Grouped bar chart
categories = ['MFCC', 'HuBERT']
word_accs = [results['Word-MFCC'], results['Word-HuBERT']]
sent_accs = [results['Sentence-MFCC'], results['Sentence-HuBERT']]

x = np.arange(len(categories))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, word_accs, width, label='Word-Level (1.5s)', color='#3498db')
bars2 = ax.bar(x + width/2, sent_accs, width, label='Sentence-Level (5.0s)', color='#e74c3c')

ax.set_ylabel('Accuracy', fontsize=12)
ax.set_title('Word-Level vs Sentence-Level Accent Detection', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.savefig(f'{PROJECT_DIR}/word_vs_sentence.png', dpi=300)
plt.show()

print("\n📊 Analysis:")
print(f"\nMFCC:")
print(f"  Word: {results['Word-MFCC']:.4f}")
print(f"  Sentence: {results['Sentence-MFCC']:.4f}")
print(f"  Improvement: {(results['Sentence-MFCC'] - results['Word-MFCC'])/results['Word-MFCC']*100:.1f}%")

print(f"\nHuBERT:")
print(f"  Word: {results['Word-HuBERT']:.4f}")
print(f"  Sentence: {results['Sentence-HuBERT']:.4f}")
print(f"  Improvement: {(results['Sentence-HuBERT'] - results['Word-HuBERT'])/results['Word-HuBERT']*100:.1f}%")

## Conclusion

### Expected Findings:
- **Sentence-level > Word-level**: More context leads to better accuracy
- **HuBERT benefits more**: Deep features leverage longer context better
- **Word-level still viable**: Can achieve reasonable accuracy with short segments

### Typical Results:
- Sentence-level: 5-15% higher accuracy than word-level
- HuBERT shows larger improvement with longer context
- MFCC more stable across durations

### Implications:
- **For real-time systems**: Word-level may be preferred (lower latency)
- **For accuracy**: Sentence-level is better
- **Trade-off**: Context vs. speed
- **Minimum duration**: At least 1-2 seconds needed for reliable detection

### Practical Applications:
- **Short utterances** (e.g., voice commands): Word-level sufficient
- **Conversations**: Sentence-level for better accuracy
- **Streaming**: Can accumulate words into sentences dynamically